In [ ]:
"""
@Author: Shahzeb Afroze
Date: 26/03/2020
This script will be used to scrape information from the html file
"""

In [2]:
# import libraries
import requests
import bs4
import os
import pandas as pd

### Locations scraping

In [3]:
# read html file downloaded from https://mrosmow.com/locations/

with open('Locations - mrosmow.com.html', 'r') as fp:
    html_file = fp.read()
    
noStarchSoup = bs4.BeautifulSoup(html_file, 'html.parser')

# loop over index 
number_of_stores = len(noStarchSoup.select('.wpsl-store-location'))


all_stores = []
for i in range(number_of_stores):
    holder = {}
    
    store_select = noStarchSoup.select('.wpsl-store-location')[i]
    holder["Location Name"] = store_select.select('strong')[0].getText()
    holder["Full Address"] = store_select.select('span')[0].getText()
    holder["Postal Code"] = store_select.select('span')[1].getText()[-7:] # to cut the string
    
    all_stores.append(holder)
    

In [4]:
all_stores_extracted = pd.read_csv("all_stores_extracted.csv", index_col=[0])
all_stores_extracted["Search Address"] = "Osmow's Shawarma Toronto, ON " + all_stores_extracted["Postal Code"]

# need to geocode all_stores_extracted
all_stores_extracted.head()

,Full Address,Location Name,Postal Code,Search Address
0,611 Queen St W,Toronto – Queen & Bathurst (Fashion District),M5V 2B7,"Osmow's Shawarma Toronto, ON M5V 2B7"
1,351 College St,Toronto – College & Augusta (Kensington Market),M5T 1S5,"Osmow's Shawarma Toronto, ON M5T 1S5"
2,497 Danforth Ave,Toronto – Danforth & Logan,M4K 1P5,"Osmow's Shawarma Toronto, ON M4K 1P5"
3,2439 Yonge St,Toronto – Yonge & Eglinton,M4P 2E7,"Osmow's Shawarma Toronto, ON M4P 2E7"
4,605 Rogers Rd,York – Rogers & Keele,M6M 1B9,"Osmow's Shawarma Toronto, ON M6M 1B9"


### Looking up place_id

In [ ]:
import googlemaps
from datetime import datetime

YOUR_API_KEY = '' # Follow instruction from https://github.com/m-wrzr/populartimes in order to find your API key

gmaps = googlemaps.Client(key=YOUR_API_KEY)

def retreive_place_id(address):
    geocode_result = gmaps.geocode(address)
    place_id = geocode_result[0]['place_id']
    return place_id

all_stores_extracted["place_id"] = all_stores_extracted['Search Address'].apply(retreive_place_id)
all_stores_extracted.to_csv("updated_file.csv")
